In [1]:
# encoding=utf8

# Advanced pandas

In [2]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

## Categorical Data

### Background and Motivation

In [3]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
print(values)
print(pd.unique(values))
print(pd.value_counts(values))

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object
['apple' 'orange']
apple     6
orange    2
dtype: int64


In [4]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
print(values)
print(dim)

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64
0     apple
1    orange
dtype: object


In [5]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### Categorical Type in pandas

In [7]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
print(fruits)
print(N)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
print(df)

['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']
8
   basket_id   fruit  count    weight
0          0   apple      9  0.201952
1          1  orange     10  3.224939
2          2   apple     14  3.723263
3          3   apple     10  1.456118
4          4   apple     10  2.763792
5          5  orange     11  0.517258
6          6   apple     10  3.330746
7          7   apple      4  1.273415


In [11]:
df.info()
fruit_cat = df['fruit'].astype('category')
print(fruit_cat)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
basket_id    8 non-null int64
fruit        8 non-null object
count        8 non-null int64
weight       8 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 336.0+ bytes
0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]


In [13]:
c = fruit_cat.values
print(c)
print(type(c))

[apple, orange, apple, apple, apple, orange, apple, apple]
Categories (2, object): [apple, orange]
<class 'pandas.core.categorical.Categorical'>


In [15]:
print(c.categories)
print(c.codes)

Index(['apple', 'orange'], dtype='object')
[0 1 0 0 0 1 0 0]


In [16]:
df['fruit'] = df['fruit'].astype('category')
print(df)
print(df.info())
print(df.fruit)

   basket_id   fruit  count    weight
0          0   apple      9  0.201952
1          1  orange     10  3.224939
2          2   apple     14  3.723263
3          3   apple     10  1.456118
4          4   apple     10  2.763792
5          5  orange     11  0.517258
6          6   apple     10  3.330746
7          7   apple      4  1.273415
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
basket_id    8 non-null int64
fruit        8 non-null category
count        8 non-null int64
weight       8 non-null float64
dtypes: category(1), float64(1), int64(2)
memory usage: 296.0 bytes
None
0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]


In [17]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [18]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [19]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [20]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

### Computations with Categoricals

In [21]:
np.random.seed(12345)
draws = np.random.randn(1000)
print(draws)
print(draws[:5])

[-0.2047  0.4789 -0.5194 -0.5557  1.9658  1.3934  0.0929  0.2817  0.769
  1.2464  1.0072 -1.2962  0.275   0.2289  1.3529  0.8864 -2.0016 -0.3718
  1.669  -0.4386 -0.5397  0.477   3.2489 -1.0212 -0.5771  0.1241  0.3026
  0.5238  0.0009  1.3438 -0.7135 -0.8312 -2.3702 -1.8608 -0.8608  0.5601
 -1.2659  0.1198 -1.0635  0.3329 -2.3594 -0.1995 -1.542  -0.9707 -1.307
  0.2863  0.378  -0.7539  0.3313  1.3497  0.0699  0.2467 -0.0119  1.0048
  1.3272 -0.9193 -1.5491  0.0222  0.7584 -0.6605  0.8626 -0.01    0.05
  0.6702  0.853  -0.9559 -0.0235 -2.3042 -0.6525 -1.2183 -1.3326  1.0746
  0.7236  0.69    1.0015 -0.5031 -0.6223 -0.9212 -0.7262  0.2229  0.0513
 -1.1577  0.8167  0.4336  1.0107  1.8249 -0.9975  0.8506 -0.1316  0.9124
  0.1882  2.1695 -0.1149  2.0037  0.0296  0.7953  0.1181 -0.7485  0.585
  0.1527 -1.5657 -0.5625 -0.0327 -0.929  -0.4826 -0.0363  1.0954  0.9809
 -0.5895  1.5817 -0.5287  0.457   0.93   -1.5693 -1.0225 -0.4028  0.2205
 -0.1934  0.6692 -1.649  -2.2528 -1.1668  0.3536  0.7021

In [22]:
bins = pd.qcut(draws, 4)
print(bins)

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], [-2.949, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, object): [[-2.949, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]


In [23]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
print(bins)
print(bins.codes[:10])

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]
[1 2 1 1 3 3 2 2 3 3]


In [25]:
bins = pd.Series(bins, name='quartile')
print(bins)
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
print(results)

0      Q2
1      Q3
2      Q2
3      Q2
4      Q4
5      Q4
6      Q3
7      Q3
8      Q4
9      Q4
       ..
990    Q1
991    Q4
992    Q1
993    Q4
994    Q2
995    Q3
996    Q2
997    Q1
998    Q3
999    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]
  quartile  count       min       max
0       Q1    250 -2.949343 -0.685484
1       Q2    250 -0.683066 -0.010115
2       Q3    250 -0.010032  0.628894
3       Q4    250  0.634238  3.927528


In [26]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

#### Better performance with categoricals

In [ ]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [ ]:
categories = labels.astype('category')

In [ ]:
labels.memory_usage()
categories.memory_usage()

In [ ]:
%time _ = labels.astype('category')

### Categorical Methods

In [ ]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

In [ ]:
cat_s.cat.codes
cat_s.cat.categories

In [ ]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

In [ ]:
cat_s.value_counts()
cat_s2.value_counts()

In [ ]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3
cat_s3.cat.remove_unused_categories()

#### Creating dummy variables for modeling

In [ ]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [ ]:
pd.get_dummies(cat_s)

## Advanced GroupBy Use

### Group Transforms and "Unwrapped" GroupBys

In [ ]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

In [ ]:
g = df.groupby('key').value
g.mean()

In [ ]:
g.transform(lambda x: x.mean())

In [ ]:
g.transform('mean')

In [ ]:
g.transform(lambda x: x * 2)

In [ ]:
g.transform(lambda x: x.rank(ascending=False))

In [ ]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [ ]:
g.transform(normalize)
g.apply(normalize)

In [ ]:
g.transform('mean')
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

### Grouped Time Resampling

In [ ]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})
df

In [ ]:
df.set_index('time').resample('5min').count()

In [ ]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
df2[:7]

In [ ]:
time_key = pd.TimeGrouper('5min')

In [ ]:
resampled = (df2.set_index('time')
             .groupby(['key', time_key])
             .sum())
resampled
resampled.reset_index()

## Techniques for Method Chaining

```python
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
```

```python
# Usual non-functional way
df2 = df.copy()
df2['k'] = v

# Functional assign way
df2 = df.assign(k=v)
```

```python
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())
```

```python
df = load_data()
df2 = df[df['col2'] < 0]
```

```python
df = (load_data()
      [lambda x: x['col2'] < 0])
```

```python
result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())
```

### The pipe Method

```python
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
```

```python
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))
```

```python
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
```

```python
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
```

```python
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))
```

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion